# Analysis and Comparison of Tweets: Boston Marathon Bombings vs. Hurricane Sandy

- Vicky Xie: Coding: Data Importing/Reading/Processing, Sentiment Analysis, Topic Modeling, pyLDAvis
- Riley Smith: Introduction, Topic Analysis, Visualization
- Finnley O'Rourke: Results/Findings: Topic Modeling
- Kayla Katakis: Research Questions, Dataset Descriptions, Results/Findings: Sentiment Analysis

## Import and Install Necessary Packages

In [1]:
# Install Neccesary Packages
!pip install textblob
!pip install nltk 
!pip install bokeh

# Import Necessary Libraries
import pandas as pd
import re
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.corpus import stopwords


# Downloading and Setting Up the Stopwords Package and Stemmer
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import matplotlib.colors as mcolors
import numpy as np
from sklearn.manifold import TSNE

!pip install pyLDAvis
import gensim
import pyLDAvis.gensim

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached pyLDAvis-3.4.1-py3-none-any.whl (2.6 MB)
  Using cached pandas-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'installed-files.txt'
Consider using the `--user` option or check the permissions.



ModuleNotFoundError: No module named 'pyLDAvis'

# Introduction

Our project focuses on the use of Sentiment Analysis and Natural Language Processing in order to extract data from tweets regarding the Boston Marathon Bombings and Hurricane Sandy. The datasets observed look at tweets in the aftermath of both disasters, some of which are classified as “on topic” and others that are classified as “off-topic”. The “on-topic” tweets refer directly to the bombing or the hurricane while the “off-topic” tweets are about anything else. 
 
In order to extrapolate specific types of data from the data set our group used both VaderSentiment and TextBlob which helped create Sentiment Analysis scores for the various tweets. While both VaderSentiment and TextBlob are repositories that help coders perform Sentiment Analysis there are a few key differences that create variance within the data. 

Some of the differences between the two models include sentiment intensity vs. sentiment polarity. VaderSentimet uses a rule-based analysis in order to assign an intensity score (-1 to 1) to a given dataset. In contrast TextBlob uses a few different components like part-of-speech tagging and noun phrase extraction in order to assign a given tweet a polarity score (negative, neutral, or positive). Additionally, VaderSentiment is designed to incorporate other aspects of tweets such as emojis and capitalization which helps the program discern the level of sentiment.

# Research Questions

In this project, we will perform sentiment analysis and topic modeling to answer the following questions:
- How do Vader and TextBlob differ in creating sentiment scores for on topic tweets in regards to the Boston Marathon Bombings?
- How do Vader and TextBlob differ in creating sentiment scores for on-topic tweets about man-made disasters (Boston Marathon Bombing) versus natural disasters (Hurricane Sandy)?
- What are the main topic cluster that crop up in the tweets that are ‘on-topic’ in regards to the Boston Marathon Bombings? 
- How do they compare to the ‘on-topic’ topic clusters in the Hurricane Sandy tweets?

# Dataset Description

The Boston Bombings dataset takes looks at just over 10,000 tweets that are either about the Boston Marathon bombings or about other random topics. The Boston Bombings dataset has 3 columns.

- `label` - a string that indicates whether or not the tweet/observation is about the Boston Marathon Bombings , can either be ‘on-topic’ or ‘off-topic’
- `tweet id` - a string that gives each observation a unique identifier value
- `tweet` - a string that  represents the entire text of each unique tweet

Using the describe() function, we can see the the number of values for each column (count), and how many of those values are unique (unique). Additionally, we can see the ‘top’ value, which represents the most common value , and its frequency. For this dataset, there are 10,012 tweet id’s that are all unique and 10,012 total tweets, with 9,226 of those being unique. We see that our “top” id value only has a frequency of one, which is intuitive due to the unique nature of that variable. Our “top” tweet is actually a retweet that shows up 34 times in the dataset, meaning that there is likely an original tweet that was retweeted 34 times. There are only two unique “label” values, either on-topic or off-topic, with the more common label being on-topic with a count of 5,648.

The Hurricane Sandy dataset is structured identically to the Boston Bombing dataset, with the same 3 columns. The major difference, of course, is that being “on-topic” now refers to tweets that are directly related to Hurricane Sandy as opposed to the Boston Marathon Bombings. 

Using the describe() function on the Hurricane Sandy dataframe, we can see that there is nearly an equal number of tweets, with Hurricane Sandy having 10,008 tweets, each with their own unique id. Of the 10,008 tweets, 9,431 are unique with two potential labels, either on or off topic, which is the same as before. Similarly to the Boston Bombing data frame, the "top" tweet in this dataset is in the form of a retweet with a frequency of 28, meaning there is a likely an original tweet, and its retweets come up 28 times in this dataset. The more common label in this dataset is on-topic with a count of 6,138.

## 2013 Boston Bombing DF

In [ ]:
bomb=pd.read_csv("2013_Boston_Bombings-ontopic_offtopic.csv")
bomb.describe()

## 2012 Sandy Hurricane DF

In [ ]:
sandy=pd.read_csv("2012_Sandy_Hurricane-ontopic_offtopic.csv")
sandy.describe()

# Preprocessing

Before we begin analyzing the data, we first need to clean up our `tweet` column to get rid of any unnecessary information. 
To do so, we wrote a function called clean_text() that removes URLs, hashtags, mentions, and any unnecessary whitespace from the text to make sure that only the most meaningful aspects of the text are left for analysis. 

Additionally, our research questions revolve entirely around the on-topic tweets, so we will filter our datasets to include only these tweets to make our analysis easier and more efficient. We will also drop any empty tweets from the dataset.

In [ ]:
# text cleaning
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # remove URLs
    text = re.sub(r'@\w+', '', text)  # remove mentions
    text = re.sub(r'#', '', text) # remove hashtag
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    return text.strip()

# Apply cleaning to Boston Bombing df:
bomb['clean_text'] = bomb.loc[(bomb[' label'] == 'on-topic')][' tweet'].apply(clean_text)
bomb=bomb.dropna(subset = ['clean_text'])

# Apply cleaning to Hurrican Sandy df:
sandy['clean_text'] = sandy.loc[(sandy[' label'] == 'on-topic')][' tweet'].apply(clean_text)
sandy=sandy.dropna(subset = ['clean_text'])

## Boston Bombing processed text:

In [ ]:
bomb['clean_text'].head()

## Hurricane Sandy processed text:

In [ ]:
sandy['clean_text'].head()

# Analysis

For our analysis, we chose to use Sentiment Analysis and Topic Modeling. Because we are interested in the way people tweet about both man-made and natural disasters, we felt that Sentiment Analysis would be a great way to judge how people use Twitter to either sympathize with our create positivity from these disasters. Additionally, Topic Modeling can help us understand what common themes pop up in tweets about these disasters and help us understand how people react when these events occur.

These two methods give us insight to how people communicate in the wake of disasters. Specifically, sentiment analysis gives us in-depth and statistical data to interpret initial public reactions and tones. In combination with topic modeling, which gives us the ability to see which subjects are the most popular, researchers are able to gain a more holistic view of how people communicate using Twitter.

In this research project Sentiment Analysis and Topic Modeling lay out the largest and most prevalent topics and sentiments in relation to the Boston Bombing and Hurricane Sandy. Furthermore, we will also observe the differences between TextBlob and Vader in order to see how the programs interpreted data differently.

# Sentiment Analysis

In order to carry out our Sentiment Analysis, we first had to develop a few functions that we can then apply to our two datasets using both TextBlob and Vader. 

Our first function `get_sentiment_textblob()` takes in a text value, which will be each tweet in the dataset, and then using the TextBlob package to return a sentiment score label. Any score greater than 0 is 'Positive' sentiment, less than 0 is 'Negative', and equal to 0 is 'Neutral'. Our other TextBlob function, `get_sent_textblob_cont()` takes each tweet and returns the numerical, continuous sentiment score.

We also have `get_sentiment_vader()` that works the same way and gives tweets a 'Positive' label to tweets with a score of 0.05 or higher, 'Negative' when the score is less than -0.05, and 'Neutral' if the score is between those two values. Since the Vader library uses more aspects of the tweets, such as capitalizations and emojis, we chose to give the Vader functions a slightly wider range for neutral tweets to account for the larger scope of variables that go into calculating the score.  Similarly to its TextBlob counterpart, `get_sent_vader_cont()` takes each tweet and returns its numerical, continuous sentiment score.

Then we apply all four of our functions to each of our two datasets to create dataframes that include each unique tweet and the sentiment scores and labels from each library.

In [ ]:
# Define a function for sentiment analysis using TextBlob
def get_sentiment_textblob(text):
    """
    this function takes text as the input and calculates the sentiment score of this given text.
    """
    sentiment = TextBlob(text).sentiment.polarity
    if sentiment > 0:
        return "Positive"
    elif sentiment < 0:
        return "Negative"
    else:
        return "Neutral"

# get continuous textblob sentiment scores
def get_sent_textblob_cont(text):
    return TextBlob(text).sentiment.polarity

# Define a function for sentiment analysis using NLTK's Vader
def get_sentiment_vader(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    if sentiment['compound'] > 0.05:
        return "Positive"
    elif sentiment['compound'] < -0.05:
        return "Negative"
    else:
        return "Neutral"
    
# get continuous vader sentiment scores
def get_sent_vader_cont(text):
    sia = SentimentIntensityAnalyzer()
    return sia.polarity_scores(text)['compound']

## Boston Bombing

In [ ]:
#Apply Sentiment Analysis functions to the Boston Bombings dataset
bomb['sentiment_textblob'] = bomb['clean_text'].apply(get_sentiment_textblob)
bomb['sent_textblob_cont'] = bomb['clean_text'].apply(get_sent_textblob_cont)
bomb['sentiment_vader'] = bomb['clean_text'].apply(get_sentiment_vader)
bomb['sent_vader_cont'] = bomb['clean_text'].apply(get_sent_vader_cont)
bomb[["clean_text", "sentiment_textblob", "sent_textblob_cont", "sentiment_vader", "sent_vader_cont"]]

## Hurricane Sandy

In [ ]:
#Applying Sentiment Analysis Functions to Hurricane Sandy Dataset
sandy['sentiment_textblob'] = sandy['clean_text'].apply(get_sentiment_textblob)
sandy['sent_textblob_cont'] = sandy['clean_text'].apply(get_sent_textblob_cont)
sandy['sentiment_vader'] = sandy['clean_text'].apply(get_sentiment_vader)
sandy['sent_vader_cont'] = sandy['clean_text'].apply(get_sent_vader_cont)
sandy[["clean_text", "sentiment_textblob", "sent_textblob_cont", "sentiment_vader", "sent_vader_cont"]]

## Visualization 

To analyze the difference between Vader and TextBlob in creating sentiment scores, we used bar plots that are color coded to represent the different libraries. TextBlob counts are shown in mediumturquoise, Vader in yellowgreen, and any overlapping counts are shown in green. To create these plots, we used Seaborns countplot() function to create two layers of the plot, one with the continuous scores from the TextBlob functions, and one with the continous scores from Vader. 

The data reveals that there is a significant difference in the way that each program caluculated the scores. In relation to the Boston Bombings Vader observed significantly more "Negative" sentiments than TextBlob. Conversely, TextBlob recorded a much larger sample of "Neutral" tweets. However the two programs recognized a similar amount of tweets with a "Positive" Sentiment. 

For Hurricane Sandy both Vader and TextBlob had significantly less Variane within the data. There was not a notable difference in the detection and sorting of different sentiments. Vader detected a slightly larger amount of "Positive" tweets, while TextBlob had more "Negative". But both programs had a very similar amount of responses with a "Neutral" connotation. The data is much more uniform when compaared to the Boston Bombings. It also indicates that people might be more likely the have similar opinions on natural disasters rather than man-made disasters. 

### Boston Bombing

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(x="sentiment_textblob", data= bomb , label="TextBlob", color='mediumturquoise', alpha=0.5)
sns.countplot(x="sentiment_vader", data= bomb , label="Vader", color='yellowgreen', alpha=0.5)
plt.legend()
plt.title('Distribution of Sentiment Scores from TextBlob and Vader: Boston Bombings')
plt.show()

### Hurricane Sandy

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(x="sentiment_textblob", data=sandy, label="TextBlob", color='mediumturquoise', alpha=0.5)
sns.countplot(x="sentiment_vader", data=sandy , label="Vader", color='yellowgreen', alpha=0.5)
plt.legend()
plt.title('Distribution of Sentiment Scores from TextBlob and Vader: Hurricane Sandy')
plt.show()

# Topic Modeling

Topic modeling is part of a larger group of algorithms known as 'unsupervised learning'. 'Unsupervised' because we don't provide the algorithm with predefined labels, it finds structure on its own, and 'learning' because it gets better and better as it processes more data. In order for us to execute Topic Modeling, we had to apply a few functions that will allow us to look at the ‘unsupervised learning’ aspect of Topic Modeling. Using the nltk, we are able to identify the larger theme of the tweets surrounding the Boston Bombing and Hurricane Sandy events. 
	
First, we cleaned the text in both datasets. Then, used we constructed a topic model by tweaking the two parameters and using the functions `number_topics = 5` and `number_words = 10`. Then, we created and fit the LDA model with a fixed random state to prevent changes with every update by creating `lda = LDA(n_components=number_topics, n_jobs=-1` and `lda.fit(count_data)`.
	
In order to view the topics, we created the function ‘def print_topics’, which includes the model, count_vectorizer, and the number_words. We then used Bokeh to graph a topic cluster of the five most talked about things from the events from both the Boston Bombing and Sandy Hurricane events. Because we have two different datasets, it’s important to note that we had to differentiate the code by using _boom and _nature in order to create two different visuals that weren’t the same. By using pyLDAvis we were able to see the results of what words were showing up repeatedly in all the tweets about the two separate events.

## Boston Bombing

In [ ]:
# Initialize CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english')

# We will only use the 'text_cleaned' column for our analysis
boom = bomb['clean_text']

# Fit and transform the processed titles
count_data_boom = count_vectorizer.fit_transform(boom)

# Tweak the two parameters below (use int values below 15)
number_topics = 5
number_words = 10

# Create and fit the LDA model with a fixed random state to prevent changes with every update
lda_model = LDA(n_components=number_topics, n_jobs=-1, random_state=5)
lda_model.fit(count_data_boom)

def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names_out()
    topic_word_distributions = model.components_
    for topic_idx, topic in enumerate(topic_word_distributions):
        print("\nTopic #%d:" % (topic_idx + 1))
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

print_topics(lda_model, count_vectorizer, number_words)



The data reveals that the top 5 Subjects in the Bosston Bombings are: 

Topic #1:
boston rt marathon bombing bomb suspect breaking suspects explosions news

- This topic surrounds the initial breaking news of the Boston Bombing. 
- It surrounds the actual events and actions of the day. 
- Terms like "breaking", "explosions", and "news" indicate that this event had just 
occured. 


Topic #2:
boston rt prayforboston marathon old died explosions year people running

- This topic surrounds some of the shock and sympathy in the aftermath of the attack. 
- Users shared their thoughts and prayers through the use of terms like "#prayforboston", "died", and "people".


Topic #3:
boston rt marathon suspects bombing fbi bombings prayforboston thoughts bostonmarathon

- This topic looks at the actual events following the Bombing. 
- They indicate that the public was sharing information in relation to the initial response to the attack. 
- Terms like, "suspects", "fbi", and "bombing" demonstrate that the public discourse surrounded justice and resolution. 


Topic #4:
boston rt suspect bombing marathon police bostonmarathon dead bomb suspects

- This topic is specifically focussed on the response of law enforcement. 
- It relates to how the FBI and Boston PD were attempting to locate the extremists. 
- Terms such as "suspect", "police", and "suspects" demonstrate this.


Topic #5:
boston rt bombing explosion bostonmarathon obama texas prayforboston martin right

- This topic surrounds the political response in the aftermath of the Boston Bombing. 
- It references aspects of politics through terms like "Obama" and "right".



### Visualization

In [ ]:
# Get topic weights
topic_weights = lda_model.transform(count_data_boom)

# Array of topic weights    
arr_boom = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr_boom = arr_boom[np.amax(arr_boom, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num_boom = np.argmax(arr_boom, axis=1)

# tSNE Dimension Reduction
tsne_model_boom = TSNE(n_components=2, verbose=1, random_state=5, angle=.99, init='pca')
tsne_lda_boom = tsne_model_boom.fit_transform(arr_boom)

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import Legend

# Plot the Topic Clusters using Bokeh
output_notebook()
mycolors_boom = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot_boom = figure(title="Clustering of Top {} Boston Bombing Topics".format(number_words), 
              width=900, height=700)

scatter_list_boom = []
for topic in np.unique(topic_num_boom):
    indices = np.where(topic_num_boom == topic)[0]
    scatter = plot_boom.scatter(x=tsne_lda_boom[indices, 0], 
                                y=tsne_lda_boom[indices, 1], 
                                color=mycolors_boom[topic])
    scatter_list_boom.append(scatter)

# Define topic labels
topic_labels_boom = ['Topic 1: Initial Breaking News', 'Topic 2: Public Sympathy', 
                     'Topic 3: Aftershock', 'Topic 4: Law Enforcement', 
                     'Topic 5: Politics']

# Create legend
legend_boom = Legend(items=[(label, [scatter]) for label, scatter in zip(topic_labels_boom, scatter_list_boom)],
                location='top_left')
plot_boom.add_layout(legend_boom, 'right')

show(plot_boom)

## pyLDAvis with Boston Marathon Bombing

In [ ]:
# Fit and transform the processed titles
count_data_boom = count_vectorizer.fit_transform(boom)

# Convert the count data to a gensim corpus
corpus = gensim.matutils.Sparse2Corpus(count_data_boom.T)

# Create a gensim Dictionary
dictionary = gensim.corpora.Dictionary.from_corpus(corpus, id2word=dict(zip(range(len(count_vectorizer.get_feature_names_out())), count_vectorizer.get_feature_names_out())))

# Create and fit the LDA model with gensim
lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=number_topics, random_state=5)

# Generate the pyLDAvis visualization
panel = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(panel)

### Topic Analysis

The 5 most prevalent topics on Twitter in the wake of the Boston Bombing were fairly standard. The first topic was the initial breaking news of the attack. The second topic surrounds the tragedy and the user's thoughts and prayers for those affected. The third topic is about the general response and the subsequent steps in order to be safer and help people heal. The fourth topic is specifically about law enforcement and their countermeasures to catch the suspects. The last topic surrounds the political response and issues left behind after the attack. 

It's important to note that there were significantly more "negative" tweets in regards to the Boston Bombing. Both Textblob and Vader detected a large sum of tweets that held negative sentiment. Per the data, users tended to view man made disasters more negatively than natural ones. The overall "negative" sentiment scores were much higher when compared to Hurricane Sandy.

## Hurricane Sandy

In [ ]:
# We will only use the 'text_cleaned' column for our analysis
natural = sandy['clean_text']

# Fit and transform the processed titles
count_data_natural = count_vectorizer.fit_transform(natural)

# Tweak the two parameters below (use int values below 15)
number_topics = 5
number_words = 10

# Create and fit the LDA model with a fixed random state to prevent changes with every update
lda_model = LDA(n_components=number_topics, n_jobs=-1, random_state=5)
lda_model.fit(count_data_natural)

def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names_out()
    topic_word_distributions = model.components_
    for topic_idx, topic in enumerate(topic_word_distributions):
        print("\nTopic #%d:" % (topic_idx + 1))
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

print_topics(lda_model, count_vectorizer, number_words)

The data reveals that the top 5 Subjects following Hurricane Sandy are:

Topic #1: hurricane sandy storm rt power lt going really away don

- This topic surrounds the destruction of infrastructure in the wake of the hurricane. 
- The term "power" in addition to "going away" seems to indicate that users were concerned with the lack of power.

Topic #2: hurricane rt sandy just like school new people power don

- This topic describes the effect on the day to day activities from Hurricane Sandy. 
- "People" and "school" make it seem like people were discussing the disruption of normal activities.

Topic #3: hurricane sandy rt frankenstorm stay safe new hit ny apocalypse

- This topic surrounded the potential catastrophic event when the hurricane collided with New York City. 
- It was a reaction to Hurricane Sandy when it headed inland. 
- Terms like "hit ny", "frankenstorm", "and "stay safe" indicate this.

Topic #4: hurricane rt fuck water went bout gonna house sandy irene

- This topic seems to demonstrate personal experiences and attitudes towards storms. 
- Specifically the term "f*ck" is telling that this topic was not a pleasurable talking point.

Topic #5: hurricane sandy rt east coast like safe shit new affected

- This topic relates to the general safety of those who live on the east coast. 
- Terms like "safety", "east coast", and "affected" demonstrate this.


### Visualization

In [ ]:
# Get topic weights
topic_weights = lda_model.transform(count_data_natural)

# Array of topic weights    
arr_natural = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr_natural = arr_natural[np.amax(arr_natural, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num_natural = np.argmax(arr_natural, axis=1)

# tSNE Dimension Reduction
tsne_model_natural = TSNE(n_components=2, verbose=1, random_state=5, angle=.99, init='pca')
tsne_lda_natural = tsne_model_natural.fit_transform(arr_natural)

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import Legend

# Plot the Topic Clusters using Bokeh
output_notebook()
mycolors_natural = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot_natural = figure(title="Clustering of Top {} Natural Disaster Topics".format(number_words), 
              width=900, height=700)

scatter_list_natural = []
for topic in np.unique(topic_num_natural):
    indices = np.where(topic_num_natural == topic)[0]
    scatter = plot_natural.scatter(x=tsne_lda_natural[indices, 0], 
                                y=tsne_lda_natural[indices, 1], 
                                color=mycolors_natural[topic])
    scatter_list_natural.append(scatter)

# Define topic labels
topic_labels_natural = ['Topic 1: Destruction of Infrastructure', 'Topic 2: Day to Day Effect', 
                        'Topic 3: NYC Gets Hit', 'Topic 4: Negative Personal Experiences', 
                        'Topic 5: East Coast Safety']

# Create legend
legend_natural = Legend(items=[(label, [scatter]) for label, scatter in zip(topic_labels_natural, scatter_list_natural)],
                location='top_left')
plot_natural.add_layout(legend_natural, 'right')

show(plot_natural)

### Topic Analysis

The five most popular topics surrounding Hurricane Sandy were also predictable. Generally users tended 
to focus on a lack of resources and a disruption of everday activities. Additionally, users sent prayers and warnings 
to those affected by the storm. They also aired their frustrations out and shared their negative attitude towards
the hurricane. 

In comparison with the Boston Bombings, the data reveals that the tweets contained a lower frequency of negative sentiments. Users tended to share tweets with a "positive" or "neutral" connotation. The data suggests that natural disasters are more likely to contain  positive sentiments in comparsion to man made disasters.

## pyLDAvis with Hurricane Sandy

In [ ]:
# Fit and transform the processed titles
count_data_natural = count_vectorizer.fit_transform(natural)

# Convert the count data to a gensim corpus
corpus = gensim.matutils.Sparse2Corpus(count_data_natural.T)

# Create a gensim Dictionary
dictionary = gensim.corpora.Dictionary.from_corpus(corpus, id2word=dict(zip(range(len(count_vectorizer.get_feature_names_out())), count_vectorizer.get_feature_names_out())))

# Create and fit the LDA model with gensim
lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=number_topics, random_state=5)

# Generate the pyLDAvis visualization
panel = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(panel)

# Results/Findings

## Sentiment Analysis
Taking a look at our first research question, we can see that there are major disparities in the way that TextBlob and Vader calculate sentiment scores for the Boston Bombings tweets. Vader produced nearly twice as many "negative" labels as TextBlob, while the opposite effect occurred in "neutral" tweets. The positive tweets from both libraries are much fewer in frequencies, but TextBlob and Vader label them very similarly. Since Vader has a wider scope of what gets included in calculating the score, it would be a reasonable assumption that the emojis and capitalizations used in these tweets provide more negative context in a addition to the words written. Without these non-textual cues and components, these tweets can be defined as more neutral as they are in TextBlob's processing. In other words, these disparities in how the tweets are processed can show us how important non-textual aspects of written communication can be important in understanding context.

In regards to comparing TextBlob and Vader's processing capabilities between man-made (Boston Bombing) and natural (Hurricane Sandy) disasters, we can clearly see a lot less variability between the two libraries when it comes to Hurricane Sandy. The overall trends are the same with Hurricane Sandy as they are with the Bombings: Vader produces more negative labels while TextBlob produces more positive and neutral labels. However, these differences are much less significant, the counts for labels are distribtued much more evenly. Opposite to the Boston Bombings, the most popular label was "Positive", which may imply that negative feelings to Hurricane Sandy, a natural disaster, are not as intense or prevalent as feelings toward a man-made disaster such as the Boston Bombings. Since we can connect the Boston Bombings to rhuman emotion and morality more easily than we can with Hurricane Sandy, these assumptions are logically sensible, but not able to be definitively made based on these types of analyses.

Further research could be done to investigate the causes of these discrepancies and would be a great pathway into understand how connecting disaster with humanity may influence the tweets that we post and the feelings we share when disaster strikes, but it's safe to say that TextBlob and Vader's different processing abilities give insight to how emojis can change the sentiments of tweets.


## Topic Modeling

Discussing our third research question, we found that the main topic cluster that are coming up the most in tweets that are ‘on-topic’ for the Boston Marathon Bombing are most involved during the actual events of the Bombing itself. There is indication that many Twitter users that are in the dataset were sharing information in relation to the initial attack, following specific words such as explosions, bombing, and #prayforboston. From the Topic Cluster we can see that the topics themselves start to intertwine as certain words start to mix, however, Topic 5 ends up being the center of everything surrounded by all the other topics. Leading into Topic 5 which was the political response in the aftermath of the Boston Marathon Bombing. Words such as Boston, bombing, explosion, prayforboston, Obama were coming up the most. There was also some on-topic tweets concerning the law enforcement response. Because Textblob and Vader detected such large numbers of tweets that were negative, the negative sentiment scores were much higher than the Hurricane Sandy event.

In comparison to the Sandy Hurricane event, the on-topic clusters were more focused on the the relevant terms ‘hurricane, sandy, rt’, and overall seemed less negative when ran in Textblob and Vader. However, salient terms were also pretty relevant in as terms such as f*ck was extremely used to really demonstrate just how un-pleasurable the event truly was. In comparison in terms of the Topic Cluster, Topic 4 which was the topic regarding the more salient terms was the center of the model as everything else revolved around it. Sandy and rt were still very used in the thousands, however, there was a much larger marginal for topic distribution.